##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import numpy as np
import os
import time

In [2]:
# Read, then decode for py2 compat.
text = open("/content/hello.txt", 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 76787 characters


In [3]:
# Take a look at the first 250 characters in text
print(text[:250])

3552445525303532555052431030435433250402122140311414051223221454124334311053025004333154510052055523101230142525553414352000030445122250221103443144025351125405433010540455334255303405102344310220342002431524234344133453531004435224411033430510443013


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

6 unique characters


In [5]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
print(chars)

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [7]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]'],
 [b'[UNK]', b'[UNK]', b'[UNK]']]>

In [10]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'[UNK][UNK][UNK][UNK][UNK][UNK][UNK]', b'[UNK][UNK][UNK]'],
      dtype=object)

In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(76787,), dtype=int64, numpy=array([4, 6, 6, ..., 2, 5, 1])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

3
5
5
2
4
4
5
5
2
5


In [15]:
seq_length = 10

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor([b'3' b'5' b'5' b'2' b'4' b'4' b'5' b'5' b'2' b'5' b'3'], shape=(11,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'35524455253'
b'03532555052'
b'43103043543'
b'32504021221'
b'40311414051'


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
dataset = sequences.map(split_input_target)

In [20]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'3552445525'
Target: b'5524455253'


In [21]:
# Batch size
BATCH_SIZE = 65

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(65, 10), dtype=tf.int64, name=None), TensorSpec(shape=(65, 10), dtype=tf.int64, name=None))>

In [22]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [23]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [24]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [25]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(65, 10, 7) # (batch_size, sequence_length, vocab_size)


In [26]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1792      
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  7175      
                                                                 
Total params: 3,947,271
Trainable params: 3,947,271
Non-trainable params: 0
_________________________________________________________________


In [27]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [28]:
sampled_indices

array([6, 6, 0, 1, 6, 3, 6, 2, 3, 4])

In [29]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'4040052503'

Next Char Predictions:
 b'55[UNK]0525123'


In [30]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [31]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (65, 10, 7)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(1.9465061, shape=(), dtype=float32)


In [32]:
tf.exp(example_batch_mean_loss).numpy()

7.0041738

In [34]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [33]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/training_checkpoints'
# Name of the checkpoint files
checkpoint_callback = ModelCheckpoint('model_weights.h5', save_weights_only=True)
csv_logger = CSVLogger('training_log.csv', append=True)

In [35]:
EPOCHS = 100

In [36]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback, csv_logger])

Epoch 1/100
107/107 [==============================] - 5s 11ms/step - loss: 1.8094 - accuracy: 0.1667
Epoch 2/100
107/107 [==============================] - 1s 10ms/step - loss: 1.7961 - accuracy: 0.1664
Epoch 3/100
107/107 [==============================] - 1s 10ms/step - loss: 1.7947 - accuracy: 0.1681
Epoch 4/100
107/107 [==============================] - 2s 10ms/step - loss: 1.7935 - accuracy: 0.1673
Epoch 5/100
107/107 [==============================] - 1s 10ms/step - loss: 1.7935 - accuracy: 0.1686
Epoch 6/100
107/107 [==============================] - 1s 10ms/step - loss: 1.7931 - accuracy: 0.1697
Epoch 7/100
107/107 [==============================] - 1s 10ms/step - loss: 1.7931 - accuracy: 0.1704
Epoch 8/100
107/107 [==============================] - 1s 10ms/step - loss: 1.7929 - accuracy: 0.1688
Epoch 9/100
107/107 [==============================] - 1s 10ms/step - loss: 1.7926 - accuracy: 0.1727
Epoch 10/100
107/107 [==============================] - 1s 10ms/step - loss: 1.792